In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from factor_analyzer import FactorAnalyzer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from scipy import stats
import statsmodels.api as sm
import os
import warnings
warnings.filterwarnings(action='ignore')
#파일경로
os.chdir('파일경로')

In [2]:
# 머신러닝
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
#가설검증
from factor_analyzer.factor_analyzer import calculate_kmo

In [4]:
df=pd.read_excel('train2.xlsx')
df

,Unnamed: 0,차대사람교통사고,경사도,지하철 이용객 수,CCTV 개수,보도 비율,불법주정차 단속 건수,상권 매출,양호여부
0,을지로2가_b,10,5.867,335087.48770,2,23.21,12,719402048267,0
1,남대문로2가_b,3,1.200,434185.24660,0,25.93,34,688341412264,0
2,충무로1가_b,28,1.484,304450.45210,4,25.42,51,1367766049543,0
3,충무로2가_b,17,4.607,345550.52330,4,20.17,41,1071913945062,0
4,명동1가_b,41,4.660,312812.00270,4,28.63,0,1817306815735,0
5,명동2가_b,13,6.043,436156.33150,4,21.42,12,2353092308019,0
6,저동1가_b,17,6.222,403417.81100,6,37.46,38,125795678637,0
7,이태원동_b,98,7.727,50100.24384,0,0.00,0,285793707754,0
8,성수동1가_b,110,1.000,44017.04932,14,9.82,0,41518150692,0
9,구의동_b,268,4.348,150610.95620,12,10.89,5,85400494500,0


In [5]:
#데이터 스케일링
from sklearn.preprocessing import StandardScaler
df_new=df.iloc[:,1:]
# Standard scaler
df_scale = pd.DataFrame(StandardScaler().fit_transform(df_new), columns=df_new.columns, index = df_new.index)
df_scale

,차대사람교통사고,경사도,지하철 이용객 수,CCTV 개수,보도 비율,불법주정차 단속 건수,상권 매출,양호여부
0,-0.796547,0.837247,0.585047,-0.595754,0.503561,-0.556696,0.080586,-1.0
1,-0.887954,-1.201768,1.242980,-0.764962,0.744527,-0.136788,0.031725,-1.0
2,-0.561500,-1.077688,0.381641,-0.426546,0.699346,0.187686,1.100509,-1.0
3,-0.705140,0.286752,0.654514,-0.426546,0.234247,-0.003181,0.635112,-1.0
4,-0.391744,0.309908,0.437155,-0.426546,0.983720,-0.785736,1.807669,-1.0
5,-0.757372,0.914141,1.256066,-0.426546,0.344985,-0.556696,2.650499,-1.0
6,-0.705140,0.992347,1.038708,-0.257338,1.765972,-0.060441,-0.853200,-1.0
7,0.352570,1.649882,-1.307048,-0.764962,-1.552617,-0.785736,-0.601512,-1.0
8,0.509268,-1.289148,-1.347436,0.419495,-0.682661,-0.785736,-0.985775,-1.0
9,2.572454,0.173595,-0.639734,0.250287,-0.587870,-0.690303,-0.916745,-1.0


In [6]:
kmo_model =(calculate_kmo(df_scale))

kmo_model

(array([0.80371185, 0.52071764, 0.648369  , 0.50587265, 0.59935197,
        0.44030479, 0.66225663, 0.29987557]),
 0.6028211722449185)

In [7]:
df2=pd.read_excel('test2.xlsx')
df_test=df2.iloc[:,1:]
# Standard scaler
df_scale2 = pd.DataFrame(StandardScaler().fit_transform(df_test), columns=df_test.columns, index = df_test.index)
df_scale2

,차대사람교통사고,경사도,지하철 이용객 수,CCTV 개수,보도 비율,불법주정차 단속 건수,상권 매출
0,-0.821342,1.200513,-0.687592,-0.966059,-0.809558,0.148878,-0.933875
1,-0.600353,-1.161446,0.345682,-0.600128,-1.031277,-0.492554,-0.440232
2,-1.005499,-0.607510,-0.614718,1.318980,0.253555,-1.157678,-0.809341
3,1.278052,1.200513,-0.858151,1.107553,-0.195567,1.815566,1.760797
4,1.149142,-0.632069,1.814779,-0.860346,1.782848,-0.314211,0.422652


In [8]:
import pandas as pd
import statsmodels.api as sm

# 컬럼 이름을 기준으로 독립, 종속 변수 지정
X = df_scale.iloc[:,:-1]
y = df_scale.iloc[:,-1]

model = sm.OLS(y, X).fit() # 회귀 모델 학습

model.summary() # 결과 출력

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   양호여부   R-squared (uncentered):                   0.382
Model:                            OLS   Adj. R-squared (uncentered):              0.127
Method:                 Least Squares   F-statistic:                              1.500
Date:                Wed, 14 Jun 2023   Prob (F-statistic):                       0.233
Time:                        16:31:13   Log-Likelihood:                         -28.281
No. Observations:                  24   AIC:                                      70.56
Df Residuals:                      17   BIC:                                      78.81
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
차대사람교통사고       -0.3321      0.354     -0.938      0.361      -1.079       0.415
경사도             0.2523      0.225      1.120      0.278      -0.223       0.728
지하철 이용객 수       0.5500      0.499      1.103      0.286      -0.502       1.602
CCTV 개수         0.4092      0.252      1.622      0.123      -0.123       0.941
보도 비율          -0.5232      0.446     -1.172      0.257      -1.465       0.418
불법주정차 단속 건수     0.4667      0.210      2.224      0.040       0.024       0.909
상권 매출          -0.2721      0.259     -1.050      0.308      -0.819       0.275
==============================================================================
Omnibus:                        2.074   Durbin-Watson:                   0.778
Prob(Omnibus):                  0.354   Jarque-Bera (JB):                1.181
Skew:                           0.184   Prob(JB):                        0.554
Kurtosis:                       1.978   Cond. No.                         6.14
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
#독립변수
X_train = df_scale.drop("양호여부", axis=1)
#종속변수
Y_train = df_scale["양호여부"]
#예측 대상 데이터 셋
X_test = df_scale2

In [10]:
# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
res_log=Y_pred
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

79.17

In [11]:
# Support Vector Machines
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
res_svc=Y_pred
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

75.0

In [12]:
#KNN
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
res_knn=Y_pred
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

62.5

In [13]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
res_gaussian=Y_pred
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

66.67

In [14]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
res_perceptron=Y_pred
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

70.83

In [15]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
res_linear_svc = Y_pred
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

79.17

In [16]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
res_decision_tree=Y_pred
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

100.0

In [17]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
res_random_forest=Y_pred
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

100.0

In [18]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
res_sgd=Y_pred
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

70.83

In [19]:
#각 모델별 점수와 양호여부 예측
models = pd.DataFrame({
    '모델': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    '점수': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree],
    '광장동': [res_svc[0], res_knn[0], res_log[0], 
              res_random_forest[0], res_gaussian[0], res_perceptron[0], 
              res_sgd[0], res_linear_svc[0], res_decision_tree[0]],
    '군자동': [res_svc[1], res_knn[1], res_log[1], 
              res_random_forest[1], res_gaussian[1], res_perceptron[1], 
              res_sgd[1], res_linear_svc[1], res_decision_tree[1]],
    '능동':   [res_svc[2], res_knn[2], res_log[2], 
              res_random_forest[2], res_gaussian[2], res_perceptron[2], 
              res_sgd[2], res_linear_svc[2], res_decision_tree[2]],
    '자양동': [res_svc[3], res_knn[3], res_log[3], 
              res_random_forest[3], res_gaussian[3], res_perceptron[3], 
              res_sgd[3], res_linear_svc[3], res_decision_tree[3]],
    '화양동': [res_svc[4], res_knn[4], res_log[4], 
              res_random_forest[4], res_gaussian[4], res_perceptron[4], 
              res_sgd[4], res_linear_svc[4], res_decision_tree[4]],
    
})
models.sort_values(by='점수', ascending=False)

,모델,점수,광장동,군자동,능동,자양동,화양동
3,Random Forest,100.00,1.0,-1.0,-1.0,1.0,1.0
8,Decision Tree,100.00,1.0,1.0,-1.0,1.0,1.0
2,Logistic Regression,79.17,1.0,-1.0,-1.0,1.0,-1.0
7,Linear SVC,79.17,1.0,-1.0,-1.0,1.0,-1.0
0,Support Vector Machines,75.00,1.0,-1.0,-1.0,1.0,-1.0
5,Perceptron,70.83,1.0,1.0,1.0,1.0,-1.0
6,Stochastic Gradient Decent,70.83,1.0,-1.0,1.0,1.0,-1.0
4,Naive Bayes,66.67,1.0,-1.0,1.0,1.0,-1.0
1,KNN,62.50,1.0,-1.0,-1.0,1.0,-1.0


In [20]:
models.sum()

모델     Support Vector MachinesKNNLogistic RegressionR...
점수                                                704.17
광장동                                                  9.0
군자동                                                 -5.0
능동                                                  -3.0
자양동                                                  9.0
화양동                                                 -5.0
dtype: object

In [21]:
coeff_df = pd.DataFrame(df_scale.columns.delete(7))
coeff_df.columns = ['독립변수']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,독립변수,Correlation
5,불법주정차 단속 건수,1.138063
3,CCTV 개수,0.529624
1,경사도,0.368307
2,지하철 이용객 수,0.263293
6,상권 매출,-0.315173
4,보도 비율,-0.320233
0,차대사람교통사고,-0.435043
